In [ ]:
## Setup code for the notebook

%matplotlib notebook 
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt      
import rasterio       
import tsd            
import utils         
import vistools     
import mvalab
import pandas as pd
import json
from scipy import ndimage
from shapely.geometry import shape, Polygon
from pyproj import Proj
import warnings
from datetime import datetime
from registration import fft_shift, hanning_win
from scipy import ndimage
import glob

cmap = plt.cm.jet
warnings.filterwarnings('ignore')

# Mise en place 

### Choix d'un aoi et calcul de son aire

In [ ]:
# enter manually aoi 
aoi = {'coordinates': [[(-64.59876276763663, -9.52084640253337), 
                        (-64.45702835936353, -9.52084640253337), 
                        (-64.45702835936353, -9.793349040517846), 
                        (-64.63963938244376, -9.793349040517846), 
                        (-64.7314453125, -9.752370139173285), 
                        (-64.59876276763663, -9.52084640253337)]], 'type': 'Polygon'}


'''
# or use a .geojson file

with open('map.geojson') as f:
    data = json.load(f)

aoi = data['features'][0]['geometry']
print(aoi)

'''

def area(aoi):
    """ calculates area in m^2 of a given aoi"""
    lon, lat = zip(*aoi['coordinates'][0])
    pa = Proj("+proj=aea +lat_1=37.0 +lat_2=41.0 +lat_0=39.0 +lon_0=-106.55")

    x, y = pa(lon, lat)
    cop = {"type": "Polygon", "coordinates": [zip(x, y)]}
    
    return(shape(cop).area)

print(area(aoi))


# Suivi de déforestation avec Sentinel-2 : calcul du NDVI

### Index NDVI et calcul de l'aire forestière sur une image

In [ ]:
def simple_equalization_8bit(im, percentiles=5):
    ''' 
    im is a numpy array
    returns a numpy array
    '''
    import numpy as np
    a, b = np.percentile(im, percentiles), np.percentile(im, 100 - percentiles)
    out = np.clip(im, a, b)
    out = np.round((out - a) / (b - a) * 255).astype(int)
    return out 


def get_sentinel2_NDVI(geotiff_path_B04, geotiff_path_B08):
    ''' 
    geotiff_path_B04 : path to band B04
    geotiff_path_B08 : path to band B08
    
    writes *NDVI.tif and returns NDVI numpy array
    '''
    out = []
    im = utils.readGTIFF(geotiff_path_B08)
    out.append(im)
    im = utils.readGTIFF(geotiff_path_B04)
    out.append(im)

    NDVI = (out[0] - out[1]) / (out[0] + out[1])  # (NIR - RED)/(NIR + RED)
    norm = plt.Normalize(vmin=NDVI.min(), vmax=NDVI.max())
    utils.writeGTIFF(NDVI, geotiff_path_B04[:-7] + 'NDVI.tif', geotiff_path_B04)
    
    return NDVI   


def get_sentinel2_RGB(geotiff_path_B02, geotiff_path_B03, geotiff_path_B04, percentiles=5):
    ''' 
    geotiff_path_B02 : path to band B02
    geotiff_path_B03 : path to band B03
    geotiff_path_B04 : path to band B04
    
    Returns RGB numpy array
    '''
    
    B04 = utils.readGTIFF(geotiff_path_B04)
    B04 = simple_equalization_8bit(B04, percentiles)
    B03 = utils.readGTIFF(geotiff_path_B03)
    B03 = simple_equalization_8bit(B03, percentiles)
    B02 = utils.readGTIFF(geotiff_path_B02)
    B02 = simple_equalization_8bit(B02, percentiles)
    
    out = [B04, B03, B02]
    im = np.squeeze(out,axis=(3))
    
    return im


def forest_area(geotiff_path, aoi, s=0.72):
    """
    geotiff_path : path to image
    aoi : the aoi corresponding to the image
    s : threshold
    
    Computes the surface of forest in km^2 in the given image.
    """
    
    total_surface = area(aoi)
    im = utils.readGTIFF(geotiff_path)
    vegetal = np.sum((im >= s))
    percentage = vegetal / (im.shape[0] * im.shape[1]) # number between 0 and 1

    return(percentage * total_surface / 10**6)

### Essayons.

On télécharge deux images : une en juin 2018, une en juin 2019.

In [ ]:
start_date_1 = datetime(2018, 6, 24)
end_date_1 = datetime(2018, 7, 1)

start_date_2 = datetime(2019, 6, 23)
end_date_2 = datetime(2019, 6, 26)

directory = './Sentinel-2/test/'

tsd.get_sentinel2.get_time_series(aoi, start_date_1, end_date_1, bands=["B04", "B08"],  out_dir= directory, api='scihub' )
tsd.get_sentinel2.get_time_series(aoi, start_date_2, end_date_2, bands=["B04", "B08"],  out_dir=directory, api='scihub'  )

On trie les images par dates.

In [ ]:
geotiff_paths_B04 = glob.glob(directory +'*B04.tif') 
geotiff_paths_B04.sort(key=(lambda k: (utils.acquisition_date(k))))

geotiff_paths_B08 = glob.glob(directory +'*B08.tif') 
geotiff_paths_B08.sort(key=(lambda k: (utils.acquisition_date(k))))

print('{} images found'.format(len(geotiff_paths_B04)))

On calcule les NDVI, on les enregistre dans le même repertoire et on les affiche dans une map couleur. Si vous voulez les visualiser en couleur naturelle, décommentez la ligne #vistools.display_gallery(NDVI_list, titres).

In [ ]:
for j in range(len(geotiff_paths_B08)):
    out = get_sentinel2_NDVI(geotiff_paths_B04[j], geotiff_paths_B08[j])

In [ ]:
geotiff_paths_NDVI = glob.glob(directory +'*NDVI.tif') 
geotiff_paths_NDVI.sort(key=(lambda k: (utils.acquisition_date(k))))

NDVI_list = []
titres = []
for geotiff_path in geotiff_paths_NDVI:
    
    im = utils.readGTIFF(geotiff_path)  
    NDVI_list.append(simple_equalization_8bit(im))
    titres.append(utils.acquisition_date(geotiff_path))
    
#vistools.display_gallery(NDVI_list, titres)
vistools.display_imshow(utils.readGTIFF(geotiff_paths_NDVI[0]), cmap='jet')
vistools.display_imshow(utils.readGTIFF(geotiff_paths_NDVI[1]),cmap='jet')

Même si la déforestation est assez claire, on peut la visualiser mieux sur l'image suivante : le pixels blancs sont des pixels pour lesquels le NDVI est passé d'une valeur supérieur à $s$ à une valeur inférieure à $s$.

In [ ]:
s = 0.72
a = 0.3
NDVI_1 = utils.readGTIFF(geotiff_paths_NDVI[0])
NDVI_2 = utils.readGTIFF(geotiff_paths_NDVI[1])

delta_NDVI = NDVI_1 * (NDVI_1 > s) - NDVI_2 * (NDVI_2 > s)
delta_NDVI = delta_NDVI * (delta_NDVI > a) + (1 - delta_NDVI) * (delta_NDVI > a)

im = utils.simple_equalization_8bit(delta_NDVI,1)
vistools.display_imshow(im)
plt.show()

On calcule enfin la surface forestière aux deux dates.

In [ ]:
for geotiff_path in geotiff_paths_NDVI:
    print("date :", utils.acquisition_date(geotiff_path)) 
    print("aire forestiere en km^2 :", forest_area(geotiff_path, aoi))

### On peut généraliser le calcul du NDVI à toute la région Rondônia.

Le principe est le suivant :
    
- On délimite la Rondônia par un polygone aoi_rondonia.
- Une centaine de polygones carrés vont recouvrir le aoi_rondonia : on calcule lon_max, lon_min, lat_max, lat_min de l'aoi_rondonia. 
- On passe ensuite en coordonnées utm via la fonction utils.lonlat_aoi_from_utm_bounding_box.
- On peut maintenant crée des polygones carrés de surface $50\times 50$ km$^2$. On prend leur intersection avec aoi_rondonia.

On calcule finalement l'aire déforestée sur chacun des polygones puis on somme.

Dans la cellule ci-dessous, on met en place tous les polygones qu'on affiche ensuite sur une carte.

In [ ]:
with open('rondonia.geojson') as f:
    data = json.load(f)
aoi_rondonia = data['features'][0]['geometry']

e_min, e_max, n_min, n_max, espg = utils.utm_bounding_box_from_lonlat_aoi(aoi_rondonia)
size_aoi = 50*1e3 # 50 km 

m = vistools.clickablemap(zoom=13)
footprint = aoi_rondonia
m.add_GeoJSON(footprint)
m.center = footprint[ 'coordinates'][0][0][::-1]

liste_e = np.arange(e_min, e_max, size_aoi)
liste_n = np.arange(n_min, n_max, size_aoi)

count = 0

liste_aoi = []
for e in liste_e :
    for n in liste_n :
        aoi_current = utils.lonlat_aoi_from_utm_bounding_box(e, e + size_aoi, n, n + size_aoi, espg)
        
        if Polygon(aoi_current[ 'coordinates'][0]).intersects(Polygon(aoi_rondonia[ 'coordinates'][0]))==False:
            continue
        polygon_intersection = Polygon(aoi_current[ 'coordinates'][0]).intersection(Polygon(aoi_rondonia[ 'coordinates'][0]))
        
        if polygon_intersection.type == 'MultiPolygon':
            for poly in list(polygon_intersection):
                liste_aoi.append({'coordinates': [list(poly.exterior.coords)], 'type': 'Polygon'})
                m.add_GeoJSON({'coordinates': [list(poly.exterior.coords)], 'type': 'Polygon'})
        
        else :
            aoi_current = {'coordinates': [list(polygon_intersection.exterior.coords)], 'type': 'Polygon'}
            liste_aoi.append(aoi_current)
            m.add_GeoJSON(aoi_current)
        
        count +=1
        print(count)


In [ ]:
display(m)

Dans la cellule ci-dessous, on calcule l'aire déforestée. NE PAS EXECUTER car cela demande le téléchargement de beaucoup d'images.

In [ ]:
count = 0
foret_debut = 0
foret_fin = 0
aire_totale = 0

for aoi_current in liste_aoi :
    if count >= 2: 
        break
        
    directory='./Sentinel-2/Rondonia/'+ 'aoi_{}/'.format(count)
    
    try :
        tsd.get_sentinel2.get_time_series(aoi_current, start_date_1, end_date_1, bands=["B04", "B08"],  out_dir= directory, api='scihub' )
        tsd.get_sentinel2.get_time_series(aoi_current, start_date_2, end_date_2, bands=["B04", "B08"],  out_dir=directory,api='scihub'  )

        geotiff_paths_B04 = glob.glob(directory + '*B04.tif')
        geotiff_paths_B04.sort(key=(lambda k: (utils.acquisition_date(k)))) 

        geotiff_paths_B08 = glob.glob(directory +'*B08.tif') 
        geotiff_paths_B08.sort(key=(lambda k: (utils.acquisition_date(k)))) 

        for j in range(len(geotiff_paths_B08)):
            get_sentinel2_NDVI(geotiff_paths_B04[j], geotiff_paths_B08[j])
        
        geotiff_paths_NDVI = glob.glob(directory + '*NDVI.tif')
        geotiff_paths_NDVI.sort(key=(lambda k: (utils.acquisition_date(k)))) 

        print("aoi numéro", count)
        print("aire tot", area(aoi_current)/1e6)
        print("aire forestière début", forest_area(geotiff_paths_NDVI[0], aoi_current))
        print("aire forestière fin", forest_area(geotiff_paths_NDVI[1], aoi_current))
        foret_debut += forest_area(geotiff_paths_NDVI[0], aoi_current)
        foret_fin += forest_area(geotiff_paths_NDVI[1], aoi_current)
        aire_totale += area(aoi_current)/1e6
        count += 1

    except Exception:
        print("Une erreur s'est produite. Il se peut qu'aucune image n'ait été trouvée pour l'AOI. On passe à l'AOI suivant.")
        
        count += 1
        continue
      


### Remarque sur le seuil du NDVI

Depuis le début, on a pris le seuil $s = 0.72$. Justifions ici cette valeur en faisant une simple SVM à partir de l'image NDVI ci-dessous.

In [ ]:
directory='./Sentinel-2/banque'
NDVI = utils.readGTIFF(directory + '2019-06-23_S2B_orbit_096_tile_20LLQ_L1C_band_NDVI.tif')
vistools.display_imshow(NDVI)


On constitue le dataset à partir de pixels de l'image, on calcul le modèle puis on évalue l'accuracy à partir d'un test set.

In [ ]:
points = []
labels = []

# classe 1
echantillons = list(NDVI[820:920, 480:580].flatten())
points.append(echantillons)
labels.append(list(np.ones(len(echantillons))))

echantillons = list(NDVI[1120:1200, 1420:1500].flatten())
points.append(echantillons)
labels.append(list(np.ones(len(echantillons))))

# classe 2
echantillons = list(NDVI[1650:1730, 380:460].flatten())
points.append(echantillons)
labels.append(list(np.zeros(len(echantillons))))

echantillons = list(NDVI[980:1100, 120:220].flatten())
points.append(echantillons)
labels.append(list(np.zeros(len(echantillons))))

points = np.array([item for sublist in points for item in sublist]).reshape(-1, 1)
labels = [item for sublist in labels for item in sublist]

print("Le dataset contient {} echantillons".format(len(points)))

In [ ]:
from sklearn import svm
clf = svm.SVC()
clf.fit(points, labels)

In [ ]:
clf.predict([[0.723], [0.724]])

In [ ]:
# Taux d'accuracy

# on crée le test set
points = []
labels = []

# classe 1
echantillons = list(NDVI[180:280, 640:740].flatten())
points.append(echantillons)
labels.append(list(np.ones(len(echantillons))))

echantillons = list(NDVI[1360:1440, 220:300].flatten())
points.append(echantillons)
labels.append(list(np.ones(len(echantillons))))

echantillons = list(NDVI[1700:1800, 1700:1800].flatten())
points.append(echantillons)
labels.append(list(np.ones(len(echantillons))))

# classe 2
echantillons = list(NDVI[1720:1840, 1360:1500].flatten())
points.append(echantillons)
labels.append(list(np.zeros(len(echantillons))))

echantillons = list(NDVI[480:580, 20:140].flatten())
points.append(echantillons)
labels.append(list(np.zeros(len(echantillons))))

echantillons = list(NDVI[1490:1515, 1720:1770].flatten())
points.append(echantillons)
labels.append(list(np.zeros(len(echantillons))))

points = np.array([item for sublist in points for item in sublist]).reshape(-1, 1)
labels = [item for sublist in labels for item in sublist]

erreur = 0
for (point, label) in zip(points, labels):
    if clf.predict([point]) != label :
        erreur += 1
                    
print("Accuracy is : {:4.2f} %".format((len(points) - erreur) / len(points) * 100))

# Suivi de déforestation avec Sentinel-1 : SAR Shadows.

Pour cette partie, nous avons choisi l'aoi suivant :

In [ ]:
aoi = {'coordinates': [[[-62.19305556, -9.60500000], 
                        [-62.24972222,-9.85861111], 
                        [-62.08750000,-9.89500000],
                        [-62.03083333,-9.64138889], 
                        [-62.19305556, -9.60500000]]],'type': 'Polygon'}

et nous avons téléchargé des images Sentinel-1 en polarisation vv pour cet AOI entre juin 2017 et juin 2019 avec tsd. Les images sont :
2017-06-09_S1A_orbit_083_GRD_vv.tif

2017-06-21_S1A_orbit_083_GRD_vv.tif

2018-06-16_S1A_orbit_083_GRD_vv.tif

2018-06-28_S1A_orbit_083_GRD_vv.tif

2018-07-10_S1A_orbit_083_GRD_vv.tif

2018-07-22_S1A_orbit_083_GRD_vv.tif

2018-08-03_S1A_orbit_083_GRD_vv.tif

2018-09-08_S1A_orbit_083_GRD_vv.tif

2019-01-06_S1A_orbit_083_GRD_vv.tif

2019-01-18_S1A_orbit_083_GRD_vv.tif

2019-02-11_S1A_orbit_083_GRD_vv.tif

2019-02-23_S1A_orbit_083_GRD_vv.tif

2019-03-19_S1A_orbit_083_GRD_vv.tif

2019-03-31_S1A_orbit_083_GRD_vv.tif

2019-04-12_S1A_orbit_083_GRD_vv.tif

2019-04-24_S1A_orbit_083_GRD_vv.tif

2019-05-06_S1A_orbit_083_GRD_vv.tif

2019-05-12_S1B_orbit_083_GRD_vv.tif

2019-05-18_S1A_orbit_083_GRD_vv.tif

2019-05-24_S1B_orbit_083_GRD_vv.tif

2019-05-30_S1A_orbit_083_GRD_vv.tif

2019-06-05_S1B_orbit_083_GRD_vv.tif

2019-06-11_S1A_orbit_083_GRD_vv.tif

2019-06-17_S1B_orbit_083_GRD_vv.tif

2019-06-23_S1A_orbit_083_GRD_vv.tif

On les a placées dans le dossier './Sentinel-1/'.

### Quelques fonctions requises

In [ ]:
def parabola_refine(c):
    '''
    maximum of parabolic interpolation 
    interpolate a parabola through the points 
    (-1,c[0]), (0,c[1]), (1,c[2]) 
    assuming that c[1]  >  c[0], c[2]
    
    I. E. Abdou, 
    "Practical approach to the registration of multiple frames of video images"
    Proceedings of Visual Communications and Image Processing '99; (1998); 
    doi: 10.1117/12.334685
    '''
    return (c[2]-c[0])/(2*(2*c[1]-c[0]-c[2]))


def crosscorrelation(a,b):
    '''
    returns the cross-correlation of a and b
        F^{-1} (F(a) * F(b)^*)
    '''

    if not (a.shape == b.shape):
        print ('ERROR: images not the same size')
        return 0,0

    sz = a.shape

    fa = np.fft.fft2(a)
    fb = np.fft.fft2(b)

    corrF = fa * np.conj(fb)

    corr = np.abs(np.fft.ifft2 ( corrF ))
    
    return corr
    
    
def max_correlation(a,b):
    '''
    computes the cross correlation of a and b and uses it 
    to compute and return (dx,dy) the subpixel shift between a and b 
    '''

    sz = a.shape

    corr = crosscorrelation(a,b) 
  
    # improve the interpolability of the maximum by filtering it
    corr = np.fft.ifftshift(
             scipy.ndimage.filters.gaussian_filter(
               np.fft.fftshift(corr),2))
    
    # position of the maximum 
    my, mx = np.unravel_index(np.argmax(corr.flatten()), corr.shape) 

    # subpixel refinement of the maximum
    dx = parabola_refine( corr[my, (mx+np.arange(-1,2))% sz[1]] )
    dy = parabola_refine( corr[(my+np.arange(-1,2))% sz[0], mx] )

    # determine the integer part of the displacement
    if mx > sz[1]/2:
        mx = mx - sz[1] 
    if my > sz[0]/2:
        my = my - sz[0] 

    return mx+dx, my+dy

    return dx, dy

### Chargement et traitement des images
On charge toutes les images dans le dossier sentinel1 et on les recale par max correlation.

In [ ]:
def recalage(imgs):
    '''
    Cette fonction prend en entrée une liste d'images de même taille et 
    recale toutes les images par maximum de corrélation par rapport à la première
    '''
    recal = []
    a = imgs[0]
    recal.append(a)
    for im in imgs[1:]:
        mx, my = max_correlation(a,im)
        b = fft_shift(im,mx,my)
        recal.append(b)
    return recal


A = 531.9922169781931 # Coefficient de calibration moyen pour la zone consité
# Sur des crops de taille raisonnable on peut supposer que le coefficient de calibration est constant

def gamma_bckscatt(imgs):
    '''
    Calcule le coefficient de rétropropagation gamma_0
    '''
    gammas = []
    for im in imgs:
        gam = 10*np.log10(abs(im)**2/A**2)
        gammas.append(gam)
    return gammas

In [ ]:
def check_changes(gamma_list,tau):
    '''
    Etant donné une liste d'images de backscatter gamma_0 et un threshold tau, 
    on repère quand est ce qu'on a un phénomène de deforestation
    '''
    n = len(gamma_list)
    tab_RCR = []
    change_times = [] # dates des changements
    change_loc = [] # position des changements
    for t in range(2,n-2):
        # Mb = (gamma_list[t]+gamma_list[t-1]+gamma_list[t-2])/3
        Mb = np.mean([gamma_list[t],gamma_list[t-1],gamma_list[t-2]],axis =0)
        # Ma = (gamma_list[t+1]+gamma_list[t+2])/2
        Ma = np.mean([gamma_list[t+1],gamma_list[t+2]], axis = 0)
        RCR = Ma - Mb
        if np.sum(RCR<tau)>1:
            change_times.append(t)
            loc = np.argwhere(RCR<tau)
            change_loc.append(loc)
            tab_RCR.append(RCR)
    return change_times, change_loc, tab_RCR

In [ ]:
# On load les images.
imgs = []
for i in sorted(glob.glob('./Sentinel-1/*_GRD_vv.tif')):
    imgs.append( utils.readGTIFF( i )[:,:,0].squeeze() )

for im in imgs:
    print(im.shape)
# On vérifie que toutes les images téléchargées sont de même taille (ce n'est pas toujours le cas)    

In [ ]:
#Pré-traitement des images : on les recale et on calcule le coefficient de rétropropagation gamma

imgs = recalage(imgs)
imgs = gamma_bckscatt(imgs)
vistools.display_gallery([ utils.simple_equalization_8bit(x,1) for x in imgs])


In [ ]:
tau = -6.5
times, loc , rcr = check_changes(imgs,tau)

In [ ]:
print(times)

On trouve souvent de la déforestation entre toutes les observations. En effet, on observe sur une zone très grande
et qui subit énormément de déforestation. 

In [ ]:
#On compare l'évolution du coefficient gamma entre une zone avec et sans sar shadows


shadow = np.zeros(len(imgs))
classic = np.zeros(len(imgs))
for k in range(1,len(imgs)-1):
    shadow[k] = (imgs[k][260,484]+imgs[k+1][260,484])/2
    classic[k] = (imgs[k-1][497,1210]+imgs[k][497,1210])/2
    
plt.figure(1)
plt.plot(shadow[1:len(imgs)-1], label = "SAR shadow")
plt.plot(classic[1:len(imgs)-1], label = "Non déforesté")
plt.legend()


In [ ]:
def deforest_near(tab_rcr):
    '''
    Cette fonction cherche les diminutions de gamma autour des points où il y a des SAR shadows 
    Elle retourne la carte de déforestation où tous les points déforestés entre la date de debut et de fin sont affichés.
    '''
    init = tab_rcr[0]
    n,m = init.shape
    defo = (init<-7)
    
    for l in range(len(tab_rcr)):
        delta = tab_rcr[l]
        shadow = np.argwhere(delta<-6)
        
        for k in range(len(shadow)):
            
            for i in range(4):
                
                for j in range(4):
                    if (shadow[k][0]+i<n):
                        a = shadow[k][0]+i
                        if (shadow[k][1]+j<m):
                            b = shadow[k][1]+j
                            defo[a,b] = 1*(delta[a,b]<-3.1)
                        if (shadow[k][1]-j>=0):
                            b = shadow[k][1]-j
                            defo[a,b] = 1*(delta[a,b]<-3.1)
                    if (shadow[k][0]-i>=0):
                        a = shadow[k][0]-i
                        if (shadow[k][1]+j<m):
                            b = shadow[k][1]+j
                            defo[a,b] = 1*(delta[a,b]<-3.1)
                        if (shadow[k][1]-j>=0):
                            b = shadow[k][1]-j
                            defo[a,b] = 1*(delta[a,b]<-3.1)
                                       
    return defo

In [ ]:
#On observe les changements entre 2017-2018 et 2018-2019

rcr1 = ((imgs[0]+imgs[1])/2-((imgs[2]+imgs[3])/2))
rcr2 = ((imgs[2]+imgs[3])/2-((imgs[22]+imgs[23])/2))
rcr = [rcr1,rcr2]

In [ ]:
defo = deforest_near(rcr) #carte de déforestation

In [ ]:
utils.writeGTIFF(defo, 'defo.tif')

In [ ]:
vistools.display_image(simple_equalization_8bit(defo,1))

In [ ]:
def naive_denoising(defo):
    '''
    On débruite naïvement la carte de déforestation
    Si un pixel labellisé comme déforesté est isolé on considère que c'est du bruit 
    '''
    n,m = defo.shape
    for i in range(1,n-1):
        for j in range(1,m-1):
            if defo[i,j]==1:
                if (defo[i,j+1]+defo[i-1,j]+defo[i+1,j]+defo[i,j-1])==0:
                    defo[i,j]=0
    return defo

In [ ]:
defo = naive_denoising(defo)
utils.writeGTIFF(defo, 'defo.tif')

In [ ]:
vistools.display_image(simple_equalization_8bit(defo,1))

# Couplage des deux méthodes

Dans cette partie, on compare la carte de déforestation obtenue avec Sentinel-1 juste au dessus, avec celle qu'on obtient avec Sentinel-2. On commence par calculer la carte de disparité pour Sentinel-2 pour l'aoi en question, entre juin 2017 et juin 2019.

Les deux images Sentinel-2 choisie sont : 

2017-06-15_S2A_orbit_053_tile_20LNQ_L1C_band_NDVI.tif

2019-06-20_S2B_orbit_053_tile_20LNQ_L1C_band_NDVI.tif

On affiche ensuite la superposition de deux aire trouvée.

In [ ]:
def debruitage(im, r=9, s=10, niter=3):
    """ 
    Pour débruiter une carte de déforestation. Les pixels isolés sont enlevés.
    """
    A = np.copy(im)
    for k in range(niter):
        for i in range(r,delta_NDVI.shape[0]-r):
            for j in range(r,delta_NDVI.shape[1]-r):
                if np.sum(delta_NDVI[i-int(r/2): i+int(r/2)+1, j-int(r/2):j+int(r/2)+1]) <= s:
                    A[i,j] = 0
                    
    return(A)

In [ ]:
directory='./Sentinel-2/banque/'

s = 0.72
a = 0.3
NDVI_1 = utils.readGTIFF(directory + '2019-06-20_S2B_orbit_053_tile_20LNQ_L1C_band_NDVI.tif')
NDVI_2 = utils.readGTIFF(directory + '2017-06-15_S2A_orbit_053_tile_20LNQ_L1C_band_NDVI.tif')

delta_NDVI = NDVI_2 * (NDVI_2 > s) - NDVI_1 * (NDVI_1 > s)
delta_NDVI = delta_NDVI * (delta_NDVI > a) + (1 - delta_NDVI) * (delta_NDVI > a)
delta_NDVI = debruitage(delta_NDVI)

delta_GAMMA = utils.readGTIFF(directory + 'DeltaGamma.tif')
images = [utils.simple_equalization_8bit(delta_NDVI,1), utils.simple_equalization_8bit(delta_GAMMA,1)]

A = np.array([utils.simple_equalization_8bit(delta_NDVI,1),utils.simple_equalization_8bit(delta_GAMMA,1), delta_GAMMA*0])
vistools.display_image(A)
plt.show()

On calcule enfin l'aire totale, l'aire en rouge, l'aire en vert, et l'union vert-rouge.

In [ ]:
aire_tot = area(aoi)
print("Aire totale:", aire_tot)

# aire Sentinel-1
A_1 = np.sum(delta_GAMMA)/(delta_GAMMA.shape[0]*delta_GAMMA.shape[1]) * aire_tot
print("Aire verte :", A_1)

# aire Sentinel-2
A_2 = np.sum(delta_NDVI)/(delta_NDVI.shape[0]*delta_NDVI.shape[1]) * aire_tot
print("Aire rouge :", A_2)

# aire couplée (union vert-rouge)
A_3 = 0
for i in range(delta_NDVI.shape[0]):
    for j in range(delta_NDVI.shape[1]):
        if delta_GAMMA[i,j] == 1 or delta_NDVI[i,j] == 1 :
            A_3 += 1
A_3 = np.sum(A_3)/(delta_NDVI.shape[0]*delta_NDVI.shape[1]) * aire_tot
print("Aire union :", A_3)